In this lab, we'll extend our previous Huber loss function and show how you can include hyperparameters in defining loss functions. We'll also look at how to implement a custom loss as an object by inheriting the Loss class.

In [3]:
try:
  %tensorflow_version 2.x
except:
  pass

import numpy as np
import tensorflow as tf

As before, this model will be trained on the xs and ys below where the relationship is $y = 2x-1$. Thus, later, when we test for x=10, whichever version of the model gets the closest answer to 19 will be deemed more accurate.

In [6]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

The loss argument in model.compile() only accepts functions that accepts two parameters: the ground truth (y_true) and the model predictions (y_pred). If we want to include a hyperparameter that we can tune, then we can define a wrapper function that accepts this hyperparameter.

In [7]:
def huber_loss_with_threshold(threshold):
  def huber_loss(y_true, y_pred):
    a = y_true - y_pred
    is_small_error = tf.abs(a) <= threshold
    small_error_loss = 0.5*tf.square(a)
    big_error_loss = threshold*(tf.abs(a) - 0.5*threshold)
    return tf.where(is_small_error, small_error_loss, big_error_loss)
  return huber_loss

We can now specify the loss as the wrapper function above. Notice that we can now set the threshold value. Try varying this value and see the results you get.

In [8]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=huber_loss_with_threshold(1))
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.79927]]


We can also implement our custom loss as a class. It inherits from the Keras Loss class and the syntax and required methods are shown below.

In [11]:
class HuberLoss(tf.keras.losses.Loss):
  threshold = 1
  def __init__(self, threshold):
    super().__init__()
    self.threshold = threshold

  def call(self, y_true, y_pred):
    a = y_true - y_pred
    is_small_error = tf.abs(a) <= self.threshold
    small_error_loss = 0.5*tf.square(a)
    big_error_loss = self.threshold*(tf.abs(a) - 0.5*self.threshold)
    return tf.where(is_small_error, small_error_loss, big_error_loss)

You can specify the loss by instantiating an object from your custom loss class.

In [12]:
model = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=HuberLoss(0.8))
model.fit(xs, ys, epochs=500, verbose=0)
print(model.predict([10.0]))

[[18.607426]]
